In [148]:
x = "asd.json"
[bl for bl in BLACKLIST if bl in x]

[]

In [1]:
from pathlib import Path

from scrapper.brand.mango.helper.download.MangoPaths import MangoPaths
from scrapper.util.io import Json_DB
import numpy as np
from random import shuffle

SCRAP_PATH = r"F:\workspace\fascrapper\scrap_results\mango"

In [2]:
len(list(walk_entries()))

652

In [3]:
from collections import defaultdict
from random import sample

def group_entries_by_view():
    grouped_images = defaultdict(lambda:[])
    for entry in walk_entries():
        images = entry["images"]
        for idx, image in enumerate(images):
            ref_idx = 1 if idx != 1 else 0
            assert ref_idx != idx
            view_name = image["description"]
            path, path_ref = images[idx]["path"], images[ref_idx]["path"]
            grouped_images[view_name].append({"path": path, "path_ref": path_ref})
    return grouped_images
grouped_entries = group_entries_by_view()

In [4]:
import ipyplot
from PIL import Image

def _prepare_images(paths, key):
    paths = [Path(SCRAP_PATH + x[key]) for x in paths]
    paths = [str(x.resolve()) for x in paths]
    paths = [Image.open(x) for x in paths]
    return paths
    
def _walk_samples(max_size=15):
    for view_name, images in grouped_entries.items():
        samples = min(max_size, len(images))
        random_examples = sample(images, samples)    

        paths = _prepare_images(random_examples, "path")    
        paths_ref = _prepare_images(random_examples, "path_ref")

        yield view_name, paths, paths_ref
        
_samples_iter = list(_walk_samples(30))

In [ ]:
view_name, paths, paths_ref = _samples_iter[21]
print(view_name)
ipyplot.plot_images(paths, 
                        labels=[f"{view_name}-{idx}" for idx in range(len(paths))], 
                        max_images=None, img_width=200)
    

In [140]:
def __flatten(a):
    return [item for sublist in a for item in sublist]

def _self_reference(indices):
    for c, idx in enumerate(indices):
        if c == idx:
            return True
    return False

def _random_references(num_idxs, max_retries=150):
    random_idxs = list(range(num_idxs))
    for _ in range(max_retries):        
        shuffle(random_idxs) # in place
        if not _self_reference(random_idxs):
            break
    return np.array(random_idxs) #copy -> None otherwise
  
    
def preprocess_negative_entries(anchor_items):
    negative_items = np.array(anchor_items)
    shuffle(negative_items)
    
    def __clean_entry(entry):
        return [{**x, 'id': entry["id"]} for x in entry["images"]]
    

    assert len(anchor_items) == len(negative_items)
    negative_items = [__clean_entry(x) for x in negative_items]
    negative_flat = __flatten(negative_items)
    
    shuffle(negative_flat)
    
    return negative_flat

def build_positive_anchors(entry):
    images_anchor, anchor_id = entry["images"], entry["id"]
    positive_idxs = _random_references(len(images_anchor))
    images_positives = [images_anchor[idx] for idx in positive_idxs]
    return [{"anchor": {**a, "id": anchor_id}, "positive": {**p, "id": anchor_id}} for (a, p) in zip(images_anchor, images_positives)]

def preprocess_positive_entries(entries):
    return __flatten([build_positive_anchors(entry) for entry in entries])        

def _validate_triplets(data):
    for d in data:
        a_id, p_id, n_id = d["anchor"]["id"], d["positive"]["id"], d["negative"]["id"]
        if a_id != p_id or a_id == n_id:
            raise Exception(f"Invalid Entry: {d}")

def build_triplets():
    anchor_items = (list(filterd_entries()))
    negative_items = np.array(anchor_items)
    shuffle(negative_items)

    assert len(anchor_items) == len(negative_items)
    
    negative_items = preprocess_negative_entries(anchor_items)
    positive_items = preprocess_positive_entries(anchor_items)
    assert len(negative_items) == len(positive_items)
    
    negative_idx = ""
    data = []
    for row in positive_items:
        anchor_id, positive_id = row["anchor"]["id"], row["positive"]["id"]
        negative_anchor = None
        
        while not negative_anchor:
            possible_n_anchor = negative_items.pop()
            if possible_n_anchor["id"] != anchor_id:
                negative_anchor = possible_n_anchor
            elif len(negative_items) < 2: #for the random chance of only the same id beeing left
                break
            else:
                negative_items.insert(0,possible_n_anchor)
                
        data.append({**row, "negative":negative_anchor})
    _validate_triplets(data)
    return data


tripplets = build_triplets()

In [144]:
list(zip(triplet_df.a_path, triplet_df.n_path, triplet_df.p_path))

[('/S20/17025956_99_B.jpg',
  '/S20/17055952_61_B.jpg',
  '/outfit/S20/17025956_99-99999999_01.jpg'),
 ('/S20/17025956_99_R.jpg',
  '/S20/17017878_94_B.jpg',
  '/S20/17025956_99_B.jpg'),
 ('/S20/17025956_99.jpg',
  '/outfit/S20/17045948_37-99999999_01.jpg',
  '/S20/17025956_99_R.jpg'),
 ('/outfit/S20/17025956_99-99999999_01.jpg',
  '/outfit/S20/17040057_45-99999999_01.jpg',
  '/S20/17025956_99.jpg'),
 ('/S20/17002023_05.jpg', '/S20/87048256_26_R.jpg', '/S20/17002023_05_R.jpg'),
 ('/outfit/S20/17002023_05-99999999_01.jpg',
  '/S20/87016325_02.jpg',
  '/S20/17002023_05_B.jpg'),
 ('/S20/17002023_05_R.jpg', '/S20/17012528_02_R.jpg', '/S20/17002023_05.jpg'),
 ('/S20/17002023_05_B.jpg',
  '/outfit/S20/17010996_91-99999999_01.jpg',
  '/outfit/S20/17002023_05-99999999_01.jpg'),
 ('/S20/17034018_05_B.jpg',
  '/outfit/S20/17053775_01-99999999_01.jpg',
  '/S20/17034018_05.jpg'),
 ('/S20/17034018_05.jpg', '/S20/17003795_30_R.jpg', '/S20/17034018_05_B.jpg'),
 ('/outfit/S20/17034018_05-99999999_01.j

In [ ]:
anchors = [x for x in tripplets]

a_views = []

In [125]:
tripplets[8]

{'anchor': {'view': 'Artikel ohne Model',
  'path': '/S20/17034018_05_B.jpg',
  'id': 'damen/shirts---tops-langarm/nahtloses-langarm-t-shirt_17034018.html'},
 'positive': {'view': 'Mittlere Ansicht',
  'path': '/S20/17034018_05.jpg',
  'id': 'damen/shirts---tops-langarm/nahtloses-langarm-t-shirt_17034018.html'},
 'negative': {'view': 'Allgemeine Ansicht',
  'path': '/outfit/S20/17053775_01-99999999_01.jpg',
  'id': 'herren/sweatshirts-ohne-kapuze/sweatshirt-mit-kangurutasche-und-schriftzug_17053775.html'}}

In [104]:
a = [0, 1, 2]
print(a.pop())
print(a)

2
[0, 1]


In [105]:
negative_items[0]

{'view': 'Rückseite des Artikels',
 'path': '/S20/17001130_02_R.jpg',
 'name': 'madchen/t-shirts/gemustertes-t-shirt-mit-knotendetail_17001130.html'}

In [102]:
positive_items[0]

{'anchor': {'view': 'Allgemeine Ansicht',
  'path': '/outfit/S20/17025956_99-99999999_01.jpg',
  'id': 'damen/shirts---tops-shirts/t-shirt-mit-rippbundchen_17025956.html'},
 'positive': {'view': 'Rückseite des Artikels',
  'path': '/S20/17025956_99_R.jpg',
  'id': 'damen/shirts---tops-shirts/t-shirt-mit-rippbundchen_17025956.html'}}